In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sys
sys.path.append('../src')
from torch import nn
from omegaconf import OmegaConf
from torch.utils.data import DataLoader
import torch
from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import StandardScaler
import numpy as np
from tqdm import tqdm
pd.options.display.max_columns=100
from multiprocessing import Pool,cpu_count
from sklearn.metrics import pairwise_distances

In [2]:
from datalib import VentilatorData,VentilatorData2,VentilatorDataClassification
import modellib
from utils import fc
import datalib
from model import LinBnReLu

In [3]:
DATA_DIR = '/mnt/disks/extra_data/kaggle/ventilator_prediction/'
R_MAP = {5: 0, 50: 1, 20: 2}
C_MAP = {20: 0, 50: 1, 10: 2}

In [4]:
train = pd.read_csv(os.path.join(DATA_DIR,'train.csv'))
test = pd.read_csv(os.path.join(DATA_DIR,'test.csv'))

In [5]:
oof_preds = pd.read_csv('../data/oof_preds.csv')

In [6]:
test_preds = pd.read_csv('v61_median.csv')

In [7]:
test_preds.rename(columns={'pressure':'preds'},inplace=True)

In [8]:
test = test.merge(test_preds[['id','preds']],on='id',how='left')

In [9]:
test.head()

,id,breath_id,R,C,time_step,u_in,u_out,preds
0,1,0,5,20,0.000000,0.000000,0,6.259305
1,2,0,5,20,0.031904,7.515046,0,5.907794
2,3,0,5,20,0.063827,14.651675,0,7.173232
3,4,0,5,20,0.095751,21.230610,0,7.665347
4,5,0,5,20,0.127644,26.320956,0,9.141693


In [10]:
(pd.read_csv('v61_median.csv')['pressure']==test['preds']).value_counts()

True    4024000
dtype: int64

In [11]:
test.shape

(4024000, 8)

In [6]:
oof_preds.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,preds,bidc
0,1,1,20,50,0.000000,0.083334,0,5.837492,5.837492,0
1,2,1,20,50,0.033652,18.383041,0,5.907794,5.837492,1
2,3,1,20,50,0.067514,22.509278,0,7.876254,7.946556,2
3,4,1,20,50,0.101542,22.808822,0,11.742872,11.813174,3
4,5,1,20,50,0.135756,25.355850,0,12.234987,12.305289,4


In [8]:
train = train.merge(oof_preds[['id','preds']],on='id',how='left')

In [9]:
train.shape

(6036000, 9)

In [10]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,preds
0,1,1,20,50,0.000000,0.083334,0,5.837492,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794,5.837492
2,3,1,20,50,0.067514,22.509278,0,7.876254,7.946556
3,4,1,20,50,0.101542,22.808822,0,11.742872,11.813174
4,5,1,20,50,0.135756,25.355850,0,12.234987,12.305289


In [13]:
train.columns

Index(['id', 'breath_id', 'R', 'C', 'time_step', 'u_in', 'u_out', 'pressure',
       'preds'],
      dtype='object')

In [14]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,preds
0,1,1,20,50,0.000000,0.083334,0,5.837492,5.837492
1,2,1,20,50,0.033652,18.383041,0,5.907794,5.837492
2,3,1,20,50,0.067514,22.509278,0,7.876254,7.946556
3,4,1,20,50,0.101542,22.808822,0,11.742872,11.813174
4,5,1,20,50,0.135756,25.355850,0,12.234987,12.305289


In [15]:
train.to_csv('../data/train_oof.csv',index=False)

In [12]:
test.to_csv('../data/test_v61.csv',index=False)

In [23]:
test.head()

,id,breath_id,R,C,time_step,u_in,u_out,preds
0,1,0,5,20,0.000000,0.000000,0,6.259305
1,2,0,5,20,0.031904,7.515046,0,5.907794
2,3,0,5,20,0.063827,14.651675,0,7.173232
3,4,0,5,20,0.095751,21.230610,0,7.665347
4,5,0,5,20,0.127644,26.320956,0,9.141693
